In [ ]:
import math
import time

import PIL.Image
import ipywidgets as widgets
import torch
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms
import traitlets
from IPython.display import display
import cv2

from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot


In [ ]:
model_road = torchvision.models.resnet18(pretrained=False)
model_road.fc = torch.nn.Linear(512, 2)
model_road.load_state_dict(torch.load('best_steering_model_xy.pth'))

model_collision = torchvision.models.alexnet(pretrained=False)
model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 2)
model_collision.load_state_dict(torch.load('best_model.pth'))

model_signs = torchvision.models.alexnet(pretrained=False)
model_signs.classifier[6] = torch.nn.Linear(model_signs.classifier[6].in_features, 3)
model_signs.load_state_dict(torch.load('sign_detection_model.pth'))
#@todo kreuzung model

In [1]:
device = torch.device('cuda')
model_road = model_road.to(device)
model_collision = model_collision.to(device)
model_signs = model_signs.to(device)
#@todo kreuzung model

NameError: name 'torch' is not defined

In [ ]:
mean = torch.Tensor([0.485, 0.456, 0.406]).half().float().to(device)
std = torch.Tensor([0.229, 0.224, 0.225]).half().float().to(device)


def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess_2(image):
    global device, normalize
    x_2 = image
    x_2 = cv2.cvtColor(x_2, cv2.COLOR_BGR2RGB)
    x_2 = x_2.transpose((2, 0, 1))
    x_2 = torch.from_numpy(x_2).float()
    x_2 = normalize(x_2)
    x_2 = x_2.to(device)
    x_2 = x_2[None, ...]
    return x_2

def preprocess_3(image):
    lelnfovnfonosfso = 222
    # fuer kreuzung @todo

In [ ]:
camera = Camera.instance(width=224, height=224, fps=10)

In [ ]:
image_widget = widgets.Image()
image_vis_widget = widgets.Image()

speed_output = widgets.Text(
    value='',
    placeholder='',
    description='(l,r):',
    disabled=True
)

model_output = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Model:',
    disabled=False
)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

In [ ]:
# Road Following sliders
speed_control_slider = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_kd_gain_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_kd_gain_slider, steering_bias_slider)

#Collision Avoidance sliders
blocked_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stopduration_slider = widgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop')
blocked_threshold = widgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

sign_verbot_slider = widgets.FloatSlider(description='signVERBOT', min=0.0, max=1.0, orientation='vertical')
sign_limit_slider = widgets.FloatSlider(description='signLIMIT', min=0.0, max=1.0, orientation='vertical')
sign_stop_slider = widgets.FloatSlider(description='signSTOP', min=0.0, max=1.0, orientation='vertical')
kreuzung_slider = widgets.FloatSlider(description='Kreuzung', min=0.0, max=1.0, orientation='vertical')

toggle_exec_btn = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    button_style='success',
    tooltip='Description',
    icon='check'
)
display(toggle_exec_btn)

display(widgets.HBox([image_widget, image_vis_widget]))
display(speed_output)

display(widgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))
display(widgets.HBox([sign_stop_slider, sign_limit_slider, sign_verbot_slider, kreuzung_slider]))

In [ ]:
speed_control, steer_gain, steer_kd_gain, steer_bias = speed_control_slider.value, 0.0, 0.0, 0.0

In [ ]:
robot = Robot()

In [ ]:
a_slider = widgets.FloatSlider(min=-math.pi, max=math.pi, step=0.01, description='a')
xy_out = widgets.Text()
display(a_slider, xy_out)

In [ ]:
def calculate_speed(last_a: float, x_in: float, y_in: float) -> (float, float, float):
    global a_slider
    a = math.atan2(x_in, y_in)
    a_slider.value = a
    xy_out.value = f"({x:.02f} {y:.02f})"
    pid = a * steer_gain + (a - last_a) * steer_kd_gain
    steer_val = pid + steer_bias
    left = max(min(speed_control + steer_val, 1.0), 0.0)
    right = max(min(speed_control - steer_val, 1.0), 0.0)
    return a, left, right

In [ ]:
def get_collision_chance(image) -> float:
    collision = model_collision(image)
    collision_softmax = functional.softmax(collision, dim=1).flatten()
    return float(collision_softmax[0])

In [ ]:
def get_road_direction(image) -> (float, float):
    model_xy = model_road(image).detach().float().cpu().numpy().flatten()
    model_x = model_xy[0]
    model_y = (0.5 - model_xy[1]) / 2.0
    return model_x, model_y
def get_motor_ratio_L2R():
    return robot.left_motor.value / robot.right_motor.value

# Beispiel: offset ist 0.02
#            left_motor.value ist 0.15
#            right_motor.value ist 0.11
#    --> L2R ist 1.37 --> left 0.12, right 0.08
def dec_motor_speed_according_to_ratio(the_motor, l2r, offset):
    temp_speed = the_motor.value - (offset + 2 * offset * the_motor.value * l2r )
    if temp_speed < 0:
        temp_speed = 0
    return temp_speed

In [1]:
angle, angle_last, stop_counter = 0.0, 0.0, 0
stop_counter_limit = 10
x, y = 0.0, 0.0
frame_counter = 0

speed_default = 0.15
speed_prev_left = 0.15
speed_prev_right = 0.15
general_threshhold = 0.8
saw_kreuzung = False
saw_sign_stop = False
saw_sign_limit = False
saw_sign_verbot = False
is_blocked = False
num_turns_verbot = 0
num_blocks = 0
num_rueckwaerts = 0

nichtmehr_kreuzung = False
nichtmehr_Stop = False
nichtmehr_Limit = False
nichtmehr_Verbot = False
nichtmehr_blocked = False


def execute(change):
    global angle, angle_last, blocked_slider, robot, stop_counter, stop_counter_limit, x, y, blocked_threshold
    global speed_control, steer_gain, steer_kd_gain, steer_bias, frame_counter
    global speed_default, speed_prev_right, speed_prev_left, general_threshhold, saw_kreuzung,saw_sign_stop, saw_sign_limit, saw_sign_verbot, is_blocked, num_turns_verbot, num_blocks, num_rueckwaerts, nichtmehr_kreuzung, nichtmehr_Stop, nichtmehr_Limit, nichtmehr_Verbot, nichtmehr_blocked


    image = change['new']
    image_preproc = preprocess(image).to(device)
    x_2 = image
    x_2 = preprocess_2(x_2)
    y_2 = model_signs(x_2)
    y_2 = functional.softmax(y_2, dim=1)

    #@todo NICHT ALLES IN JEDEM FRAME
    #@todo flags setzen, in anderer Task behandeln, nicht auf einmal
    #@todo kreuzung model

    # get slider values
    steer_gain = steering_gain_slider.value
    steer_kd_gain = steering_kd_gain_slider.value
    steer_bias = steering_bias_slider.value
    stop_counter_limit = stopduration_slider.value
    max_speed = speed_control_slider.value

    prob_blocked = get_collision_chance(image_preproc)
    blocked_slider.value = prob_blocked

    prob_sign_verbot = float(y_2.flatten()[0])
    prob_sign_limit = float(y_2.flatten()[1])
    prob_sign_stop = float(y_2.flatten()[2])

    sign_verbot_slider.value = prob_sign_verbot # + prob_not_blocked - 1 # result should always be 0
    sign_limit_slider.value = prob_sign_limit
    sign_stop_slider.value = prob_sign_stop

    prob_kreuzung = 0.0# @todo prob_kreuzung = float(y_3.flatten()[])
    kreuzung_slider.value = prob_kreuzung

    '''
    Verbale Beschreibung, was das Programm tun soll:

    Wenn der Roboter blockiert ist und die Flag noch nicht gesetzt wurde,
    Flag setzen und schrittweise abbremsen.
    Im Stillstand bis zu 20 Frames warten, danach Rückwärtsgang einlegen.

    Flag bei Kreuzung setzen und langsamer werden, nach Kreuzung schneller werden (unideal bei Kurven aber egal)

    Bei Speedlimits ggf. langsamer werden. Speedlimits gelten bis zur nächsten Kreuzung.

    Bei Stopschildern Flag setzen und bei der nächsten Kreuzung anhalten, 3s warten.

    Bei Durchfahrtverbotsschild 90° im Uhrzeigersinn (rechts) drehen. Bei Kreuzung weiterfahren,
    ansonsten 180° gegen den Uhrzeigersinn drehen. Bei Kreuzung weiterfahren,
    ansonsten 90° gegen den Uhrzeigersinn drehen und wieder zurück fahren

    Im Falle, dass der Roboter nicht im Rückwärtsgang ist oder sich bei Durchfahrtverbot dreht oder warten muss, den Road-Following Koordinaten (PoI) folgen

    Flags werden entsprechend wieder zurückgesetzt (hoffentlich)
    '''

    ### COLLAVOIDANCE ###
    if num_rueckwaerts > 1:
        robot.backward(1) # volle Kanne Rückwärtsgang (nur kurz)
        time.sleep(0.05)

    if prob_blocked > general_threshhold:
        if is_blocked:
            num_blocks += 1
            if num_blocks >= 20:
                num_rueckwaerts += 1
                num_blocks = 0
        else:
            is_blocked = True
            num_blocks += 1
            if (robot.left_motor.value <= 0.11) and (robot.right_motor.value <= 0.11):
                robot.stop()
            else: # schrittweise abbremsen
                speed_prev_left = robot.left_motor.value
                speed_prev_right = robot.right_motor.value
                robot.left_motor.value *= 0.75
                robot.right_motor.value *= 0.75

    else:
        if num_rueckwaerts > 0:
            num_rueckwaerts = 0
        if is_blocked: # wenn es 3 mal anzeigt dass er blockiert ist obwohl gemessen frei, Block aufheben
            num_blocks += 1
            if num_blocks > 3:
                is_blocked = False
                num_blocks = 0

    #else:
     #   x, y, speed_control = 0.0, 0.0, 0
    if not is_blocked and (num_rueckwaerts == 0):
        ### Kreuzung ###
        if prob_kreuzung > general_threshhold:
            saw_kreuzung = True
            if robot.right_motor.value > 0.11 or robot.left_motor.value > 0.11:
                speed_prev_left = robot.left_motor.value
                speed_prev_right = robot.right_motor.value
                robot.left_motor.value = 0.11
                robot.right_motor.value = 0.11
        else:
            if saw_kreuzung:
                saw_kreuzung = False
                speed_prev_left = robot.left_motor.value
                speed_prev_right = robot.right_motor.value
                robot.left_motor.value = 0.15
                robot.right_motor.value = 0.15

        ### SIGN LIMIT ###
        if prob_sign_limit > general_threshhold:
            saw_sign_limit = True
            speed_prev_left = robot.left_motor.value
            speed_prev_right = robot.right_motor.value
            robot.left_motor.value = 0.13
            robot.right_motor.value = 0.13
        if saw_sign_limit and saw_kreuzung:
            # tempolimit gilt nicht mehr, aber langsam bei Kreuzung
            saw_kreuzung = False
            saw_sign_limit = False
            robot.left_motor.value = 0.11
            robot.right_motor.value = 0.11

        ### SIGN VERBOT ### (rechts versuchen, ggf links versuchen, ansonsten zurück fahren)
        if num_turns_verbot == 2:
            if not saw_kreuzung:
                robot.left(0.3)
                time.sleep(0.5)
            else:
                saw_kreuzung = False
            num_turns_verbot = 0
            robot.left_motor.value = 0.11
            robot.right_motor.value = 0.11
            x, y = get_road_direction(image_preproc) # nicht mehr blockiert --> Berechnung

        elif num_turns_verbot == 1:
            if not saw_kreuzung:
                num_turns_verbot += 1
                robot.left(0.3)
                time.sleep(1)
                robot.stop()
            else:
                saw_kreuzung = False
                num_turns_verbot = 0
                robot.left_motor.value = 0.11
                robot.right_motor.value = 0.11
                x, y = get_road_direction(image_preproc) # nicht mehr blockiert --> Berechnung
        elif prob_sign_verbot > general_threshhold:
           # saw_sign_verbot = True # wird nicht verwendet
           num_turns_verbot += 1
           robot.right(0.3)
           time.sleep(0.5)
           robot.stop()
        else:
            ### Stop sign ###
            if prob_sign_stop > general_threshhold:
                saw_sign_stop = True
            if saw_sign_stop:
                if saw_kreuzung:
                    robot.stop()
                    time.sleep(3)
                    saw_sign_stop = False
                    saw_kreuzung = False
                    robot.left_motor.value = 0.11
                    robot.right_motor.value = 0.11
            x, y = get_road_direction(image_preproc) # nicht mehr blockiert --> Berechnung

        angle, left, right = calculate_speed(angle_last, x, y)
        angle_last = angle

        robot.left_motor.value = left
        robot.right_motor.value = right

        speed_output.value = f'{left:.02f}/{right:.02f}'
    '''
    if frame_counter % camera.fps == 0:
        frame_counter = 0

        snapshot = image.copy()
        snapshot = cv2.circle(snapshot, (int(x * snapshot.width), int(y * snapshot.height)), 8, (0, 255, 0), 3)
        image_vis_widget.value = bgr8_to_jpeg(snapshot)
    frame_counter += 1
    '''

execute({'new': camera.value})

IndentationError: expected an indented block (574240453.py, line 81)

In [ ]:
def start_stop_toggle(obj):
    if obj["new"]:
        toggle_exec_btn.button_style = 'warning'
        toggle_exec_btn.icon = 'warning'
        toggle_exec_btn.description = "Stop"
        camera.observe(execute, names='value')
    else:
        toggle_exec_btn.button_style = 'success'
        toggle_exec_btn.icon = 'check'
        toggle_exec_btn.description = 'Start'
        camera.unobserve(execute, names='value')
        time.sleep(0.1)
        robot.stop()


toggle_exec_btn.observe(start_stop_toggle, 'value')